In [ ]:
# https://colab.research.google.com/drive/1zki5smRQDDIYGZ9LuTzVLV_qcq4dH8Aj#scrollTo=nltkxvCS7wl_

In [1]:
!pip install torch 
!pip install peft
!pip install bitsandbytes
!pip install transformers
!pip install trl 
!pip install accelerate
!pip install einops

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [3]:
# Model
base_model = "microsoft/phi-2"
new_model = "phi-2-medquad"

# Dataset
dataset = load_dataset("prsdm/medquad-phi2-1k", split="train")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map={"": 0}
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# LoRA configuration
peft_config = LoraConfig(
    r= 64,          
    lora_alpha= 16,
    lora_dropout=0.05, #0.1
    bias="none",
    task_type="CAUSAL_LM",
   #target_modules= ["Wqkv", "out_proj"] #["Wqkv", "fc1", "fc2" ] # ["Wqkv", "out_proj", "fc1", "fc2" ]
)

In [6]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 1,
    fp16 = False,
    bf16 = False,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 1,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_32bit",
    lr_scheduler_type = "cosine",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 25,
)

In [7]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length= None,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Train model
trainer.train()

c:\Users\Wen-Liang\AppData\Local\Programs\Python\Python312\Lib\site-packages\trl\trainer\sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

  0%|          | 0/250 [00:00<?, ?it/s]

c:\Users\Wen-Liang\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
c:\Users\Wen-Liang\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\phi\modeling_phi.py:712: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 1.3598, 'grad_norm': 0.33330702781677246, 'learning_rate': 0.0001975746552556772, 'epoch': 0.1}
{'loss': 1.2094, 'grad_norm': 0.28896892070770264, 'learning_rate': 0.00018550053929480202, 'epoch': 0.2}
{'loss': 1.0873, 'grad_norm': 0.2637534439563751, 'learning_rate': 0.00016449948488669639, 'epoch': 0.3}
{'loss': 1.0479, 'grad_norm': 0.28253138065338135, 'learning_rate': 0.000136764169663272, 'epoch': 0.4}
{'loss': 1.1101, 'grad_norm': 0.18904398381710052, 'learning_rate': 0.00010519038181318999, 'epoch': 0.5}
{'loss': 1.0509, 'grad_norm': 0.23014378547668457, 'learning_rate': 7.307467669163655e-05, 'epoch': 0.6}
{'loss': 1.0504, 'grad_norm': 0.2241186797618866, 'learning_rate': 4.377019014049223e-05, 'epoch': 0.7}
{'loss': 1.0419, 'grad_norm': 0.248228058218956, 'learning_rate': 2.03365443542764e-05, 'epoch': 0.8}
{'loss': 1.0651, 'grad_norm': 0.21027523279190063, 'learning_rate': 5.22039891260262e-06, 'epoch': 0.9}
{'loss': 1.0, 'grad_norm': 0.2374918907880783, 'learning_ra

TrainOutput(global_step=250, training_loss=1.1022884826660155, metrics={'train_runtime': 7239.2855, 'train_samples_per_second': 0.138, 'train_steps_per_second': 0.035, 'total_flos': 1.270197576732672e+16, 'train_loss': 1.1022884826660155, 'epoch': 1.0})

In [8]:
# Save trained model
trainer.model.save_pretrained(new_model)

c:\Users\Wen-Liang\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
prompt = "What are the treatments for Gastrointestinal Carcinoid Tumors?"
instruction = f"### Instruction: {prompt} "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\Wen-Liang\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


 ### Assistant: Treatment for gastrointestinal carcinoid tumors depends on the type of tumor, its stage, and whether it has spread. The following treatments may be used:  - Surgery to remove the tumor.  - Chemotherapy to kill cancer cells.  - Radiation therapy to kill cancer cells.  - Targeted therapy to kill cancer cells.  - Clinical trials.  - Supportive care to manage symptoms and side effects.  - Palliative care to relieve symptoms and improve quality of life.  - Support groups.  - Complementary and alternative medicine.  - Targeted therapy is a type of treatment that uses drugs or other substances to identify and attack cancer cells. These substances are called targeted agents or targeted therapy. Targeted therapy is a type of treatment that uses drugs or other substances to identify and attack cancer cells. These substances are called targeted agents or targeted therapy. Targeted therapy
